In [118]:
#!pip install -U python-dotenv
#!pip install spotipy

In [1]:
import pandas as pd
import os
import plotly.express as px
import spotipy
from spotipy import oauth2
from dotenv import load_dotenv
import os
from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
path = os.getcwd()

# setting up authentication for spotipy
load_dotenv(dotenv_path=path + f"\env_vars.env")
os.environ['SPOTIPY_CLIENT_ID'] = os.environ.get('SPOTIPY_CLIENT_ID')
os.environ['SPOTIPY_CLIENT_SECRET'] = os.environ.get('SPOTIPY_CLIENT_SECRET')
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

### Part 1: Loading in the Data
#### We will load in my personal data blah blah, you can replace the string with your file path to analyze your music

In [4]:
num_histories = 4
listening = pd.DataFrame()
for i in range(num_histories):
    temp_df = pd.read_json(path + f"\MyData\StreamingHistory{i}.json")
    listening = pd.concat([listening, temp_df])
print(listening.shape)
listening.head(5)

(33284, 4)


,endTime,artistName,trackName,msPlayed
0,2020-12-26 00:02,Michael Schulte,The Love You Left Behind,194546
1,2020-12-26 00:05,Lewis Capaldi,Lost On You,197178
2,2020-12-26 00:08,Arlissa,Praying for Love,182535
3,2020-12-26 00:12,JP Saxe,A Little Bit Yours,225680
4,2020-12-26 00:16,My Brothers And I,Just Getting to Know You,252840


In [5]:
playlists = pd.read_json(path + f"\MyData\Playlist1.json")
playlists.head(5)

,playlists
0,"{'name': 'My Playlist #23', 'lastModifiedDate'..."
1,"{'name': 'My Playlist #22', 'lastModifiedDate'..."
2,"{'name': 'Stealing Spotify Daily Mixes', 'last..."
3,"{'name': 'Jess Chill Reccomended by Spotify', ..."
4,"{'name': 'Jess <3 Vibes?', 'lastModifiedDate':..."


### Part 2: EDA

In [6]:
playlists['playlists'][3]

{'name': 'Jess Chill Reccomended by Spotify',
 'lastModifiedDate': '2021-12-04',
 'items': [{'track': {'trackName': 'House',
    'artistName': 'Joshua Moss',
    'albumName': 'House',
    'trackUri': 'spotify:track:3BdtiKSZveePCDhfncBOFd'},
   'episode': None,
   'localTrack': None},
  {'track': {'trackName': 'Apple Pie',
    'artistName': 'Lizzy McAlpine',
    'albumName': 'Give Me A Minute',
    'trackUri': 'spotify:track:5FKNq0lLJnP3iZzBTD0PZj'},
   'episode': None,
   'localTrack': None},
  {'track': {'trackName': 'Free',
    'artistName': 'Elina',
    'albumName': 'Free',
    'trackUri': 'spotify:track:3hOWsTM9OpI6c72iQHP4EB'},
   'episode': None,
   'localTrack': None},
  {'track': {'trackName': 'Pancakes for Dinner',
    'artistName': 'Lizzy McAlpine',
    'albumName': 'Give Me A Minute',
    'trackUri': 'spotify:track:4HffeEF97c6UxNePgbuECW'},
   'episode': None,
   'localTrack': None}],
 'description': None,
 'numberOfFollowers': 0}

In [57]:
# information based on a single song
track_name, track_artist = 'Hey There Delilah', 'Plain White T\'s'
track_uri = sp.search(q='artist:' + track_artist + ' track:' + track_name, type='track')['tracks']['items'][0]['uri']
# track_analysis = sp.audio_analysis(track_uri) # too much info for me rn
track_features = sp.audio_features(track_uri)
track_features[0]

{'danceability': 0.656,
 'energy': 0.291,
 'key': 2,
 'loudness': -10.572,
 'mode': 1,
 'speechiness': 0.0293,
 'acousticness': 0.872,
 'instrumentalness': 0,
 'liveness': 0.114,
 'valence': 0.298,
 'tempo': 103.971,
 'type': 'audio_features',
 'id': '4RCWB3V8V0dignt99LZ8vH',
 'uri': 'spotify:track:4RCWB3V8V0dignt99LZ8vH',
 'track_href': 'https://api.spotify.com/v1/tracks/4RCWB3V8V0dignt99LZ8vH',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4RCWB3V8V0dignt99LZ8vH',
 'duration_ms': 232533,
 'time_signature': 4}

In [55]:
counts = listening.query('msPlayed >= 3000').groupby(by=['trackName', 'artistName']).size().reset_index()
counts = counts.rename(columns={0: 'count'}).sample(n=10, weights='count', random_state=42)#.query('count >= 5').sort_values('count', ascending=False) # either listened to >5 times or top x songs or random x songs
counts#.to_csv('grouped.csv')

,trackName,artistName,count
2092,I Wish,Declan J Donovan,12
5201,friendship?,Jordy Searcy,10
3966,Sorry That I Do That,Anna Clendening,60
3232,One Day,Lovejoy,3
848,Complicated,Olivia O'Brien,6
298,Astronaut Kids,Hotel Fiction,1
4757,We Could Be Anything (Black & White),Jay Ollero,18
3249,Only Everything - Acoustic,Quinn Lewis,7
3822,Singing in the Rain,Aimee Vant,34
103,About the Rain,Ron Pope,15


In [56]:

lst_of_cols = list(track_features[0].keys())
# lst_of_cols.remove(['type', 'analysis_url'])
# lst_of_cols.remove(x for x in ['type', 'analysis_url'])
[lst_of_cols.remove(col) for col in ['type', 'uri', 'analysis_url', 'track_href']]
def find_id(song):
    # print(song)
    try:
        song_dict = sp.search(q='artist:' + song[1] + ' track:' + song[0], type='track')
        song_uri = song_dict['tracks']['items'][0]['uri']
        track_features = sp.audio_features(song_uri)[0]
        return pd.Series([track_features[i] for i in lst_of_cols])
    except:
        return pd.Series(['DNE'] * len(lst_of_cols))
    
    
counts[lst_of_cols] = counts[['trackName', 'artistName']].apply(find_id, axis=1)
counts

,trackName,artistName,count,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
2092,I Wish,Declan J Donovan,12,0.625,0.241,7,-9.312,1,0.0308,0.84500,0.000000,0.1530,0.0960,111.902,7sXnAth7OEXjGhrW7Qsl3Y,198720,4
5201,friendship?,Jordy Searcy,10,0.571,0.224,4,-11.704,1,0.0646,0.69600,0.000000,0.1030,0.5060,108.526,0Io5aNyPCvK31w1hffcTbC,220133,4
3966,Sorry That I Do That,Anna Clendening,60,0.714,0.404,9,-8.759,1,0.0439,0.19600,0.000001,0.2830,0.2990,98.016,2WxCovJaXPloCHw0yJ0Z0z,149821,4
3232,One Day,Lovejoy,3,0.339,0.730,10,-8.568,0,0.0358,0.00737,0.000000,0.0472,0.8370,183.144,58U47mHjyLwUwJZ7voUt15,144375,4
848,Complicated,Olivia O'Brien,6,0.657,0.355,7,-12.047,1,0.0715,0.69400,0.000425,0.1230,0.0808,74.990,226PUvIuxGdceSijXXZRRz,189000,4
298,Astronaut Kids,Hotel Fiction,1,0.287,0.425,9,-7.893,1,0.0322,0.61000,0.000015,0.1190,0.1370,82.252,1LIaXS82jVxkZQdXbfM5QH,280322,4
4757,We Could Be Anything (Black & White),Jay Ollero,18,0.672,0.414,6,-7.919,1,0.0408,0.83200,0.000000,0.1510,0.4290,75.969,4Ej5jSHuUcfUNselqiV1qV,221613,4
3249,Only Everything - Acoustic,Quinn Lewis,7,0.600,0.337,2,-8.516,1,0.0304,0.77800,0.000000,0.0686,0.6830,131.977,65SCuNx59yFP9hRWRtcrkv,174545,3
3822,Singing in the Rain,Aimee Vant,34,0.650,0.749,0,-10.647,1,0.0924,0.55900,0.000445,0.0864,0.2880,125.020,2CFxEzF1PTUeGuTrI3yVO5,246240,4
103,About the Rain,Ron Pope,15,0.642,0.705,4,-7.750,1,0.0347,0.36400,0.000006,0.1580,0.3400,106.043,4mWVFz0wjFUIvUlD0ox9Ej,192533,4


### Part 3: Analyzing the Data
#### PCA/plot/etc, figuring out what matters most in a song, what makes a song similar to another?

### Part 3: Loading in a playlist and comparing it to other songs in the playlist
#### Read in, PCA, plot with colors

### Part 4: Selecting additional relevant songs for playlist
#### Find most similar ones to playlist songs